# Working With Geospatial Data in Python

### Chad Hawkins

# What Are Geospatial Data?

* Simply put, data that have a geographic component

# What is GIS?

* Geographic Information Systems
* Systems for storing, displaying and interacting with geospatial data

# Types of Geospatial Data

* Vector
* Raster
* Tabular

# Vector Data

* Common formats include GeoJSON, Shapefile, and KML
* Point, Line, Polygon
* Could be derived from GPS or digitzed from imagery
* Uses include observations, parcels, roads, soils, administrative boundaries

# GeoJSON Example

<br>
```
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [-77.410133, 39.413948]
      },
      "properties": {
        "name": "IronNet Office"
      }
    }
  ]
}
```

# KML Example

<br>
```
<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">

  <Placemark>

    <name>A simple placemark on the ground</name>

    <Point>
			<coordinates>8.542952335953721,47.36685263064198,0</coordinates>
    </Point>

  </Placemark>

</kml>
```

# Soils

![alt text](images/soils.png "Soils")

# Raster Data

* Common formats include GeoTIFF, Erdas Imagine, and GRIB
* Used primarily for remotely sensed data (satellite imagery, weather, elevation)
* Continuous or categorical data
* Efficient for both rendering and storing

# Landcover Classification (Categorical)

![alt text](images/landcover_classification.png "Landcover Classification Raster")

# Elevation (Continuous)

![alt text](images/elevation.png "Elevation Raster")

# Tabular Data

* Data such as addresses and zip codes
* Not geospatial in the strictest terms but correspond to well understood geographic regions
* Addresses are translated to real world coordinates using a process called Geocoding

# Map Projections

* We are flattening a spheroid; We cannot preserve every attribute in doing so
* Compromises must be made to shape, area, distance, and direction depending on the analysis
* In this presentation with are working with latitude and longitude (unprojected data)
* As such, we cannot compute meaningful areas for example
* We can compute distances that are good enough however (Haversine formula)

# Spatial Operations

* Intersect
* Intersection/Clip
* Within/Contains
* Buffer
* Merge
* Zonal Statistics (raster statistics for vector-defined region)
* Vectorize
* Rasterize

# Python Packages

* Shapely
* Fiona
* Rasterio
* Basemap
* Folium
* Pyproj
* GeoDjango

# Related Tools

* GDAL/OGR
* PostgreSQL/PostGIS
* QGIS
* Leaflet
* OpenLayers
* Google Earth
* Mapnik
* R
* ArcGIS

# Data Sources

* [Maryland GIS Data Catalog](https://data.imap.maryland.gov/)
* [Frederick County](https://frederickcountymd.gov/5969/Download-GIS-Data)
* [ArcGIS Open Data](https://hub.arcgis.com/pages/open-data)
* [OpenStreetMap](https://www.openstreetmap.org)
* [Census Bureau TIGER/Line](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)
* [USGS](https://earthexplorer.usgs.gov/)
* [NWS](https://www.weather.gov/gis/)

# Let's Do Some Spatial Analysis

## ... of Bigfoot sightings!

# First We Need to Acquire and Prepare Some Data

In [ ]:
from collections import defaultdict
from copy import deepcopy
import datetime
import fiona
import folium
from folium.plugins import MarkerCluster
import haversine
import pandas as pd
from pprint import pprint
import requests
from shapely.geometry import mapping, Point, shape

In [ ]:
url = 'https://opendata.arcgis.com/datasets/d0afc5b29e4346cc9a4cf8e43bcaaed0_0.geojson'

response = requests.get(url)
data = response.json()

print(data['type'])

pprint(data['features'][0])

In [ ]:
# Let's convert TimeWhen to a datetime
timewhen_format = '%Y-%m-%d %H:%M:%S'

valid_sightings = []
sighting_times = []
for feature in deepcopy(data['features']):
    # Ignore sightings with bad or missing time observations
    try:
        sighting_time = datetime.datetime.strptime(
            feature['properties']['TimeWhen'], 
            timewhen_format
        )
    except (TypeError, ValueError):
        continue
    feature['properties']['TimeWhen'] = sighting_time
    valid_sightings.append(feature)
    sighting_times.append(sighting_time)

In [ ]:
print('Total sightings: {}'.format(len(data['features'])))
print('Valid sightings: {}'.format(len(valid_sightings)))
print('Oldest sighting: {}'.format(min(sighting_times)))
print('Most recent sighting: {}'.format(max(sighting_times)))

In [ ]:
# Let's restrict our map to more recent sightings
recent_sightings = [
    sighting for sighting in valid_sightings if 
    sighting['properties']['TimeWhen'] >= datetime.datetime(2010, 1, 1)
]

print('Recent sightings: {}'.format(len(recent_sightings)))

# Let's Look at These Data on a Map

In [ ]:
# Let's start with an empty map
# Create a function so we can reuse later
def get_map_object():
    return folium.Map(
        location=[44, -103],
        tiles='Stamen Terrain',
        zoom_start=4
    )

m = get_map_object()

In [ ]:
m

In [ ]:
m = get_map_object()

for feature in recent_sightings:
    lon, lat = feature['geometry']['coordinates']
    folium.Marker(
        [lat, lon],
    ).add_to(m)

In [ ]:
m

# That's a Bit Overwhelming...

In [ ]:
# Recreate the map using clusters

m = get_map_object()

locations = []
for feature in recent_sightings:
    lon, lat = feature['geometry']['coordinates']
    locations.append((lat, lon))
cluster = MarkerCluster(locations=locations)

m.add_child(cluster)

In [ ]:
m

# Much Better... Let's Customize a Bit

In [ ]:
m = get_map_object()

locations = []
popups = []
icons = []
for feature in recent_sightings:
    lon, lat = feature['geometry']['coordinates']
    locations.append((lat, lon))
    # Show the Description field when clicked
    popup = folium.Popup(feature['properties']['Descr'], max_width=200)
    popups.append(popup)
    # Use a custom icon
    icon = folium.map.Icon(color='gray', icon='paw', prefix='fa')
    icons.append(icon)

cluster = MarkerCluster(
    locations=locations, 
    popups=popups, 
    icons=icons, 
    name='Bigfoot Sightings'
)

m.add_child(cluster)
folium.LayerControl().add_to(m)

In [ ]:
m

# Let's Count the Sightings by State

* Points on a map can be difficult to properly interpret
* It's often useful to aggregate in some fashion
* In this case, by state seems reasonable
* First we need to associate the point data with a state

In [ ]:
states = []

# Open the shapefile with fiona
path = 'tl_2018_us_state/tl_2018_us_state.shp'
with fiona.open(path) as shp:
    pprint(shp.meta)
    for record in shp:
        # Store the shapely object for the geometry so we can do our comparison
        record['geometry'] = shape(record['geometry'])
        states.append(record)

In [ ]:
sightings_by_state = defaultdict(int)

for sighting in recent_sightings:
    location = shape(sighting['geometry'])
    # Loop over the states until we find one that contains our sighting
    # This is VERY inefficient, especially with larger datasets
    # The proper way to do this is to use an rtree index 
    # (beyond the scope of this presentation)
    for state in states:
        if state['geometry'].contains(location):
            state_abbr = state['properties']['STUSPS']
            sightings_by_state[state_abbr] += 1
            break

# Create a dataframe
df = pd.DataFrame(
    [(state_abbr, count) for state_abbr, count in sightings_by_state.items()],
    columns=['State', 'Count']
)

df.sort_values(by='Count', ascending=False).head()

In [ ]:
# Create GeoJSON
for state in states:
    # We simplify the geometry here because our boundaries 
    # are far more detailed than required
    geometry = state['geometry'].simplify(0.003, preserve_topology=True)
    state['geometry'] = mapping(geometry)
    
states_geojson = {
    'type': 'FeatureCollection', 
    'features': states
}

# Let's Create a Choropleth Map

* "A thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map" [1]
* In this case the variable is the number of sightings for each state

[1]: https://en.wikipedia.org/wiki/Choropleth_map

In [ ]:
m = get_map_object()

folium.Choropleth(
    geo_data=states_geojson,
    name='Sightings by State',
    data=df,
    columns=['State', 'Count'],
    key_on='feature.properties.STUSPS',
    fill_color='YlOrRd',
    legend_name='Number of Sightings',
    nan_fill_opacity=0.9,
    nan_fill_color='white',
    fill_opacity=0.9
).add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
m

# Let's Find the Closest Sighting

In [ ]:
ironnet_location = (39.413948, -77.410133)

closest_distance = None
closest_sighting = None

for sighting in valid_sightings:
    lon, lat = sighting['geometry']['coordinates']
    sighting_location = (lat, lon)
    # Get the distance in miles
    distance = haversine.haversine(
        ironnet_location, 
        sighting_location, 
        unit='mi'
    )
    if closest_distance is None:
        closest_distance = distance
        closest_sighting = sighting
    elif distance < closest_distance:
        closest_distance = distance
        closest_sighting = sighting

desc = closest_sighting['properties']['Descr']
print(
    'Closest sighting was {} miles away on {}\n{}'.format(
        round(closest_distance, 2), 
        closest_sighting['properties']['TimeWhen'].date().isoformat(),
        desc[desc.index('Report'):desc.index('</b>')]
    )
)

# Questions?

* Email: cwh@chadwhawkins.com
* GitHub: @chadwhawkins
* Frederick Tech Slack